In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from unity_environment_wrapper import unity_env, str_numbers
from Multi_Agent_DDPG import Multi_MA_DDPG


In [2]:
def fix_state6(self,state,prev_state=None):
    state = np.reshape(state,(3,8))[:,0:6]
    if prev_state is None:
        state[0,:] = state[2,:]
        state[1,:] = state[2,:]
    else:
        state[0,:] = prev_state[6:12]
        state[1,:] = prev_state[12:]
        #if abs(state[2,0]-state[1,0]) < 1e-6:
        #    state[2,2] = 0.0
        #if abs(state[2,1]-state[1,1]) < 1e-6:
        #    state[2,3] = 0.0
    return state.reshape(-1)

def fix_state4(self,state,prev_state=None):
    state = np.reshape(state,(3,8))[:,[0,1,4,5]]
    if prev_state is None:
        state[0,:] = state[2,:]
        state[1,:] = state[2,:]
    else:
        state[0,:] = prev_state[4:8]
        state[1,:] = prev_state[8:]
    return state.reshape(-1)

def modify_batch6(self):
    if self.duplicate_batch > 1:
        ind_start = self.batch_size
        ind_end = self.batch_size*2
        self.states[ind_start:ind_end,:,[4,10,16]] *= -1
        self.next_states[ind_start:ind_end,:,[4,10,16]] *= -1
        self.states[ind_start:ind_end,[0,1],:]      = self.states[ind_start:ind_end,[1,0],:]
        self.actions[ind_start:ind_end,[0,1],:]     = self.actions[ind_start:ind_end,[1,0],:]
        self.next_states[ind_start:ind_end,[0,1],:] = self.next_states[ind_start:ind_end,[1,0],:]
        if self.rewards.shape[1] > 1:
            self.rewards[ind_start:ind_end,[0,1]]   = self.rewards[ind_start:ind_end,[1,0]]
        if self.dones.shape[1] > 1:
            self.dones[ind_start:ind_end,[0,1]]     = self.dones[ind_start:ind_end,[1,0]]

def modify_batch4(self):
    if self.duplicate_batch > 1:
        ind_start = self.batch_size
        ind_end = self.batch_size*2
        self.states[ind_start:ind_end,:,[2,6,10]] *= -1
        self.next_states[ind_start:ind_end,:,[2,6,10]] *= -1
        self.states[ind_start:ind_end,[0,1],:]      = self.states[ind_start:ind_end,[1,0],:]
        self.actions[ind_start:ind_end,[0,1],:]     = self.actions[ind_start:ind_end,[1,0],:]
        self.next_states[ind_start:ind_end,[0,1],:] = self.next_states[ind_start:ind_end,[1,0],:]
        if self.rewards.shape[1] > 1:
            self.rewards[ind_start:ind_end,[0,1]]   = self.rewards[ind_start:ind_end,[1,0]]
        if self.dones.shape[1] > 1:
            self.dones[ind_start:ind_end,[0,1]]     = self.dones[ind_start:ind_end,[1,0]]

def modify_batch4_abs(self):
    np.abs(self.states[:,:,[2,6,10]], out=self.states[:,:,[2,6,10]])
    np.abs(self.next_states[:,:,[2,6,10]], out=self.next_states[:,:,[2,6,10]])
    if self.duplicate_batch > 1:
        ind_start = self.batch_size
        ind_end = self.batch_size*2
        self.states[ind_start:ind_end,[0,1],:]      = self.states[ind_start:ind_end,[1,0],:]
        self.actions[ind_start:ind_end,[0,1],:]     = self.actions[ind_start:ind_end,[1,0],:]
        self.next_states[ind_start:ind_end,[0,1],:] = self.next_states[ind_start:ind_end,[1,0],:]
        if self.rewards.shape[1] > 1:
            self.rewards[ind_start:ind_end,[0,1]]   = self.rewards[ind_start:ind_end,[1,0]]
        if self.dones.shape[1] > 1:
            self.dones[ind_start:ind_end,[0,1]]     = self.dones[ind_start:ind_end,[1,0]]

    

In [3]:
env = unity_env(file_name="Tennis_Windows_x86_64/Tennis.exe", no_graphics=True, score_goal=0.5, verbose_level=1,\
                set_modify_state=fix_state4)

    
agent = Multi_MA_DDPG(state_shape = env.state_shape, action_size = env.action_size, num_agents=env.num_agents,\
                       seed=1,\
                       replay_buffer_size = int(1e6),\
                       replay_batch_size =128,\
                      duplicate_batch=1,\
                      set_modify_batch=modify_batch4_abs,\
                       update_every=4,\
                       update_times=1,\
                       lr_actor=1e-5,\
                       lr_critic=1e-6,\
                       noise_sigma=0.2,\
                       noise_theta=0.1,\
                       no_reward_value = -0.02,\
                       actor_arch = ['b',256,'b','r',256,'b','r',128,'b','r'],\
                       critic_arch = [['b',256,'b','r',256,'b','r',128],['b',128,'b','r',64],['b','r',256,'b','r',128,'b','r',1]],\
                       gamma = 0.95,\
                       use_cuda=True, verbose_level=2)

output_name = "results"

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Exploring Unity environment...
Selected brain name:  TennisBrain
Selected brain:       Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 
Number of actions:    2
Number of agents:     2
States have shape:    (12,)
State of agent 0 look like:
 [ -6.638  -1.500   6.001   6.000  -6.638  -1.500   6.001   6.000  -6.638  -1.500   6.001   6.000]
State of agent 1 look like:
 [ -7.079  -1.500  -6.001   6.000  -7.079  -1.500  -6.001   6.000  -7.079  -1.500  -6.001   6.000]
Initializing DDPG_Agent with PyTorch device named: cuda:0
replay_buffer_size = 1000000
replay_batch_size  = 128
seed               = 1
gamma              = 0.95
tau                = 0.001
update_every       = 4
update

In [ ]:
agent.reset()
all_scores, best_test_score, best_test_average, best_test_stdev = env.train(agent,output_name,num_episode_search=2500,noise_decay=0.995)


Training DDPG agent:
output_filename    = results
noise_minimal      = 0.01
noise_decay        = 0.995
num_episode_search = 2500
max_num_episodes   = 10000
score_window_size  = 100
Episode 100 curr= 0.10 	Average=0.028 Stdev=0.047 Composite=-0.019 Max=0.200 	Total=2060 Positives=32 Negatives=100 Zeros=1928 
Saving checkpoint...
Episode 102 curr= 0.10 	Average=0.029 Stdev=0.048 Composite=-0.019 Max=0.200 	Total=2107 Positives=33 Negatives=102 Zeros=1972 
Saving checkpoint...
Episode 111 curr= 0.10 	Average=0.028 Stdev=0.045 Composite=-0.017 Max=0.100 	Total=2254 Positives=34 Negatives=111 Zeros=2109 
Saving checkpoint...
Episode 115 curr= 0.09 	Average=0.029 Stdev=0.045 Composite=-0.017 Max=0.100 	Total=2347 Positives=36 Negatives=115 Zeros=2196 
Saving checkpoint...
Episode 116 curr= 0.10 	Average=0.030 Stdev=0.046 Composite=-0.016 Max=0.100 	Total=2375 Positives=37 Negatives=116 Zeros=2222 
Saving checkpoint...
Episode 333 curr= 0.10 	Average=0.033 Stdev=0.049 Composite=-0.016 Max=0.2

Episode 1785 curr= 0.10 	Average=0.069 Stdev=0.049 Composite=0.020 Max=0.200 	Total=42912 Positives=852 Negatives=1772 Zeros=40288 
Saving checkpoint...
Episode 1787 curr= 0.09 	Average=0.070 Stdev=0.048 Composite=0.022 Max=0.200 	Total=42978 Positives=853 Negatives=1774 Zeros=40351 
Saving checkpoint...
Episode 1789 curr= 0.10 	Average=0.071 Stdev=0.048 Composite=0.023 Max=0.200 	Total=43042 Positives=854 Negatives=1776 Zeros=40412 
Saving checkpoint...
Episode 1790 curr= 0.10 	Average=0.071 Stdev=0.048 Composite=0.023 Max=0.200 	Total=43074 Positives=855 Negatives=1777 Zeros=40442 
Saving checkpoint...
Episode 1791 curr= 0.10 	Average=0.071 Stdev=0.048 Composite=0.023 Max=0.200 	Total=43107 Positives=856 Negatives=1778 Zeros=40473 
Saving checkpoint...
Episode 1792 curr= 0.09 	Average=0.072 Stdev=0.048 Composite=0.025 Max=0.200 	Total=43140 Positives=857 Negatives=1779 Zeros=40504 
Saving checkpoint...
Episode 1793 curr= 0.10 	Average=0.072 Stdev=0.048 Composite=0.025 Max=0.200 	Tota

## plot the scores

In [ ]:
fig = plt.figure()
ax  = fig.add_subplot(111)
plt.plot(np.arange(len(all_scores)), all_scores)
plt.ylabel('Score')
plt.xlabel('Train Episode #')
plt.show()

In [ ]:
agent.load(output_name)
test_composite, test_mean, test_stdev, test_scores = env.test_agent_on_many_episodes(agent)
print("Final Test scores on {} episodes:".format(len(test_scores)))
print("Mean score:    ",test_mean)
print("Std.Dev. score:",test_stdev)

env.test_agent_on_single_episode(agent)